In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
stage_2_detailed_class_info = pd.read_csv("../input/rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv")
stage_2_sample_submission = pd.read_csv("../input/rsna-pneumonia-detection-challenge/stage_2_sample_submission.csv")
stage_2_train_labels = pd.read_csv("../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv")

Declaration

In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
import pandas as pd 
import glob

In [ ]:
DATA_DIR = "../input/rsna-pneumonia-detection-challenge/"
ROOT_DIR = '/kaggle/working'

In [ ]:
train_dicom_dir = os.path.join(DATA_DIR, 'stage_2_train_images')
test_dicom_dir = os.path.join(DATA_DIR, 'stage_2_test_images')

In [ ]:
df_1 = pd.merge(stage_2_detailed_class_info,stage_2_train_labels,on = 'patientId')
print(df_1.head(10))
print(df_1.shape)

Testing Connection with Data Files

In [ ]:
g_Train = glob.glob(train_dicom_dir + '/*.dcm')
g_Test = glob.glob(test_dicom_dir + '/*.dcm')

In [ ]:
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g_Train))
print ('\n'.join(g_Train[:5]))
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g_Test))
print ('\n'.join(g_Test[:5]))

In [ ]:
%%time

PathDicom = train_dicom_dir
lstFilesDCM = []  # create an empty list
FileName = []
annotations = {}
for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))
            FileName.append(filename.strip('.dcm'))

In [ ]:
df_2 = pd.DataFrame({'patientId':FileName,'path':lstFilesDCM})
print(df_2.head(10))
print(df_2.shape)

In [ ]:
df_3 = pd.merge(df_1,df_2,on = 'patientId')
print(df_3.head(10))
print(df_3.shape)

In [ ]:
image_annotations = {fp: [] for fp in lstFilesDCM}
len(image_annotations)

In [ ]:
anns = pd.read_csv(os.path.join(DATA_DIR, 'stage_2_train_labels.csv'))
print(anns.head(10))
print(anns.shape)


In [ ]:
for index, row in anns.iterrows(): 
    fp = os.path.join(PathDicom, row['patientId']+'.dcm')
    image_annotations[fp].append(row)

In [ ]:
image_annotations

In [ ]:
len(image_annotations)

In [ ]:
#pd.DataFrame.from_dict(data)
List_1 = [(k, v) for k, v in image_annotations.items()]
print(List_1[:10])
print(len(List_1))

In [ ]:
df_4 = pd.DataFrame(List_1)
print(df_4.head(10))
print(df_4.shape)

In [ ]:
df_4.rename(columns={0: "path_1", 1:"annotation"}, inplace = True)
print(df_4.head(10))
print(df_4.shape)

In [ ]:
df_4['annotation'][0][0][0]

In [ ]:
patientId_df = []
for index, row in df_4.iterrows():
    patientId_df.append(df_4['annotation'][index][0][0])
len(patientId_df)

In [ ]:
df_4['patientId'] = patientId_df
print(df_4.head(10))
print(df_4.shape)

In [ ]:
df_5 = pd.merge(df_3,df_4,on = 'patientId')
print(df_5.head(10))
print(df_5.shape)

In [ ]:
IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 1024

In [ ]:
df_5['original_height'] = IMAGE_HEIGHT

In [ ]:
df_5['original_width'] = IMAGE_WIDTH

In [ ]:
df_5.head(10)

In [ ]:
df_5.drop(['path_1'],axis=1,inplace = True)
df_5.head(10)

In [ ]:
df_5.head(10)

In [ ]:
df_5.to_csv('/kaggle/working/Data_Prep.csv')
np.save('Data_Prep.npy',df_5)

Testing Files

In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
import pandas as pd 
import glob

In [ ]:
Dataset = np.load('/kaggle/input/data-preparation/Data_Prep.npy',allow_pickle = True)

In [ ]:
Dataset.astype

In [ ]:
Dataset_DB =  pd.DataFrame(data=Dataset)

In [ ]:
Dataset_DB.shape

In [ ]:
Dataset_DB.rename(columns={0: "patientId", 1:"class",2:'x',3:'y',4:'width',5:'height',6:'Target',7:'path',8:'annotation',9:'original_height',10:'original_width'}, inplace = True)

In [ ]:
Dataset_DB.head(2)

**START**

In [ ]:
from random import randint
#Image_ID = 100
Image_ID = randint(0,29280)


In [ ]:
fb = Dataset_DB['path'][Image_ID]
image_info = pydicom.read_file(fb)
image = image_info.pixel_array
plt.imshow(image,cmap='gray')
print(Dataset_DB['patientId'][Image_ID])
print(image.shape)

In [ ]:
if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
image.shape

In [ ]:
annotation = Dataset_DB['annotation'][Image_ID]
count = len(annotation)
print(count)
if count == 0:
    mask = np.zeros((Dataset_DB['original_height'][Image_ID], Dataset_DB['original_width'][Image_ID], 1), dtype=np.uint8)
else:
    mask = np.zeros((Dataset_DB['original_height'][Image_ID], Dataset_DB['original_width'][Image_ID], count), dtype=np.uint8)
   
    for i, a in enumerate(annotation):
        if a['Target'] == 1:
            x = int(a['x'])
            y = int(a['y'])
            w = int(a['width'])
            h = int(a['height'])
            mask_instance = mask[:, :, i].copy()
            cv2.rectangle(mask_instance, (x, y), (x+w, y+h), 255, -1)
            mask[:, :, i] = mask_instance

In [ ]:
masked = np.zeros(image.shape[:2])
for i in range(mask.shape[2]):
    masked += image[:, :, 0] * mask[:, :, i]
plt.imshow(masked, cmap='gray')
plt.axis('off')

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 2, 2)
masked = np.zeros(image.shape[:2])
for i in range(mask.shape[2]):
    masked += image[:, :, 0] * mask[:, :, i]
plt.imshow(masked, cmap='gray')
plt.axis('off')

print(Image_ID)
print(fb)
print(Dataset_DB['patientId'][Image_ID])